<a href="https://colab.research.google.com/github/tikendraw/Amazon-review-sentiment-analysis/blob/main/amazon-review-sentiment-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Reviews for Sentiment Analysis

## Objective

Here we will be Building ML and DL models to predict the Polarity of reviews.
We will be performing series of experiments with different models to achieve the best classification metrics.(while not abusing the machine we have)

## About Dataset
[Dataset here](https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews)


### OVERVIEW
Contains 34,686,770 Amazon reviews from 6,643,669 users on 2,441,053 products, from the Stanford Network Analysis Project (SNAP). This subset contains 1,800,000 training samples and 200,000 testing samples in each polarity sentiment.

### ORIGIN
The Amazon reviews dataset consists of reviews from amazon. The data span a period of 18 years, including ~35 million reviews up to March 2013. Reviews include product and user information, ratings, and a plaintext review. For more information, please refer to the following paper: J. McAuley and J. Leskovec. Hidden factors and hidden topics: understanding rating dimensions with review text. RecSys, 2013.

### DESCRIPTION
The Amazon reviews polarity dataset is constructed by taking review score 1 and 2 as negative, and 4 and 5 as positive. Samples of score 3 is ignored. In the dataset, class 1 is the negative and class 2 is the positive. Each class has 1,800,000 training samples and 200,000 testing samples.

If you need help extracting the train.csv and test.csv files check out the starter code.

The files train.csv and test.csv contain all the training samples as comma-separated values.

The CSVs contain polarity, title, text. These 3 columns in them, correspond to class index (1 or 2), review title and review text.

polarity - 1 for negative and 2 for positive
title - review heading
text - review body
The review title and text are escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".

In [ ]:
# ! git clone https://github.com/tikendraw/funcyou.git -q
# ! pip install funcyou/
# ! rm -rf funcyou
# ! pip install tensorflow_hub

In [1]:

!pip install contractions -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.4 MB/s eta 0:00:00


In [1]:
import datetime
import os
import pickle
import random
import re
import sys
import tarfile
import time
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
from pathlib import Path

import contractions
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.utils import check_random_state

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, TextVectorization
from tensorflow_hub import KerasLayer

try:
    import funcyou
except ImportError:
    # Install funcyou
    !pip install git+https://github.com/tikendraw/funcyou.git@main -q

# Importing useful functions from funcyou
from funcyou.dataset import download_kaggle_resource
from funcyou.plot import distplot_axis, plot_history, compare_histories
from funcyou.preprocessing.text import IntegerVectorizer
from funcyou.sklearn.metrics import calculate_results

print('Tf version:', tf.__version__)
print('GPU:', len(tf.config.list_physical_devices('GPU')))


2023-09-01 15:39:23.112360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-01 15:39:24.445659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tf version: 2.12.1
GPU: 0


2023-09-01 15:39:27.220833: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-01 15:39:27.561068: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
def set_global_seed(seed):
    tf.random.set_seed(seed)
    global random_state
    random_state = check_random_state(seed)


# Set the seed value
SEED = 42 # should be same as seed in config file

# Call the function to set the seeds
set_global_seed(SEED)


In [3]:
# download and enter the repo to access other files

colab = 'google.colab' in str(get_ipython())

if colab:
    os.system('git clone https://github.com/tikendraw/Amazon-review-sentiment-analysis.git')

    if os.getcwd() != '/content/Amazon-review-sentiment-analysis/notebook':
        os.chdir('Amazon-review-sentiment-analysis/notebook')
    print('Current working directory: ',os.getcwd())


    # load google drive
    from google.colab import drive
    drive.mount('/content/drive')


In [4]:
cur_dir = Path(os.getcwd())
model_dir = Path(os.getcwd()).parent /'model'
data_dir = Path(os.getcwd()).parent /'dataset'


In [5]:
print(cur_dir)
print(model_dir)
print(data_dir)

/home/t/aproject/Amazon-review-sentiment-analysis/notebook
/home/t/aproject/Amazon-review-sentiment-analysis/model
/home/t/aproject/Amazon-review-sentiment-analysis/dataset


In [6]:
# Download the data if you don't have locally
dataset_url = 'https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews'

In [7]:
if colab:
    download_kaggle_resource(dataset_url, data_dir, kaggle_json_path = '/content/kaggle.json')

    from zipfile import ZipFile
    with ZipFile(data_dir/'amazon-reviews'/'amazon-reviews.zip') as f:
        f.extractall(data_dir)

# Load the data

In [8]:
#reading data
df = pl.read_csv(data_dir/'train.csv',new_columns = ['polarity', 'title','text'])  # gives TextFileReader, which is iterable with chunks of 1000 rows.


In [9]:
df.describe()

describe,polarity,title,text
str,f64,str,str
"""count""",3.599999e6,"""3599999""","""3599999"""
"""null_count""",0.0,"""0""","""0"""
"""mean""",1.5,null,null
"""std""",0.5,null,null
"""min""",1.0,"""""","""this is the b…"
"""max""",2.0,"""♦ LOVE IT ♦""","""…were Marvin a…"
"""median""",1.0,null,null
"""25%""",1.0,null,null
"""75%""",2.0,null,null


In [10]:
# check for nulls and drop if any
df.null_count()

polarity,title,text
u32,u32,u32
0,0,0


In [11]:
#drop nulls
df.drop_nulls()
print()

In [12]:
# checking for classs imbalance
df['polarity'].value_counts()

polarity,counts
i64,u32
1,1800000
2,1799999


# We will map the polarity between 0 for negative sentiment to 1 for positive sentiment

In [13]:
df = df.with_columns([
                    pl.col('polarity').apply(lambda x: 0 if x == 1 else 1).alias('polarity')
                     ])

df = df.with_columns([
                     pl.col('polarity').cast(pl.Int16, strict=False).alias('polarity')
                     ])


In [14]:
df.sample(10)

polarity,title,text
i16,str,str
0,"""Super Ape Inna…","""I picked a cop…"
0,"""MISSING A SCEN…","""Do not buy thi…"
0,"""Requires paid …","""This sounded l…"
1,"""One of the ver…","""I have read ot…"
1,"""One of the bes…","""This product i…"
1,"""This was Great…","""I relistened t…"
1,"""Sex & the Sing…","""I love movies …"
0,"""Dull Read""","""This book was …"
0,"""ATC 2K""","""The sound on t…"


## Note: We will be combining text and title columns . makes more sense.

In [15]:
df = df.with_columns([
    (pl.col('title')+' ' + pl.col('text')).alias('review')
])

df = df.with_columns([
    pl.col('review').str.to_lowercase()
])


In [16]:
# df = df.with_columns([
#     pl.col('title').apply(lambda x: len(str(x).split())).alias('title_len'),
# ])

# df = df.with_columns([
#     pl.col('text').apply(lambda x: len(str(x).split())).alias('text_len'),
# ])

# df = df.with_columns([
#     pl.col('review').apply(lambda x: len(str(x).split())).alias('review_len')
# ])


In [17]:

# plt.figure(figsize=(15,5))
# distplot_axis(df['title_len'].to_numpy())

# plt.figure(figsize=(15,5))
# distplot_axis(df['text_len'].to_numpy())

# plt.figure(figsize=(15,5))
# distplot_axis(df['review_len'].to_numpy())

In [18]:
df = df.select(['review', 'polarity'])

In [19]:
df.head()

review,polarity
str,i16
"""the best sound…",1
"""amazing! this …",1
"""excellent soun…",1
"""remember, pull…",1
"""an absolute ma…",1


In [20]:
df.write_csv(data_dir / 'preprocessed_df.csv')

In [65]:
df = pl.read_csv(data_dir/'preprocessed_df.csv')
df.head()

review,polarity
str,i64
"""the best sound…",1
"""amazing! this …",1
"""excellent soun…",1
"""remember, pull…",1
"""an absolute ma…",1


## Preprocessing text

In [22]:

# Compile the regular expressions outside the function for better performance
PUNCTUATION_REGEX = re.compile(r'[^\w\s]')
DIGIT_REGEX = re.compile(r'\d')
SPECIAL_CHARACTERS_REGEX = re.compile(r'[#,@,&]')
MULTIPLE_SPACES_REGEX = re.compile(r'\s+')

def clean_text(x: str) -> str:
    expanded_text = contractions.fix(x)  # Expand contractions
    text = PUNCTUATION_REGEX.sub(' ', expanded_text.lower())  # Remove punctuation after lowering
    text = DIGIT_REGEX.sub('', text)  # Remove digits
    # Remove special characters (#,@,&)
    text = SPECIAL_CHARACTERS_REGEX.sub('', text)
    # Remove multiple spaces with single space
    text = MULTIPLE_SPACES_REGEX.sub(' ', text)
    return text.strip()

# Compile regex patterns for better performance


In [23]:
q = "i'm going to market.you wanna come?huh?"
clean_text(q)

'i am going to market you want to come huh'

In [24]:
counter_object_filepath = model_dir / 'counter.pkl'

save_dir = Path('/content/drive/MyDrive/models/amazon_sentiment')
if colab:
    counter_object_filepath = save_dir / 'counter.pkl'


In [25]:
# a = IntegerVectorizer(preprocessing_func=clean_text, min_freq=3) # 19 min  , contraction fix takes most of the time
# a.adapt(df['review'])


In [26]:
# # save the counter object
# counter_object = a.vocab.counter  #Vocab size: 261933

# with open(counter_object_filepath, 'wb') as output_file:
#     pickle.dump(counter_object, output_file)
#     print('file saved at',counter_object_filepath)

In [27]:
with open(counter_object_filepath, 'rb') as output_file:
    counter_object = pickle.load(output_file)

In [28]:
# Get the least common words
least_common_words = counter_object.most_common()[::-1]


In [29]:
f = 100_000
counter_object.most_common()[f:f+100]

[('eclair', 14),
 ('krown', 14),
 ('pti', 14),
 ('winterbourne', 14),
 ('stracher', 14),
 ('zekk', 14),
 ('slection', 14),
 ('fiv', 14),
 ('kreh', 14),
 ('kaneda', 14),
 ('xhr', 14),
 ('funning', 14),
 ('guenivere', 14),
 ('aiki', 14),
 ('erring', 14),
 ('sundiver', 14),
 ('retirements', 14),
 ('carryin', 14),
 ('wfp', 14),
 ('lunt', 14),
 ('hydrangeas', 14),
 ('coombe', 14),
 ('precisa', 14),
 ('reeler', 14),
 ('sesson', 14),
 ('handsomest', 14),
 ('milimeter', 14),
 ('gastar', 14),
 ('suggestible', 14),
 ('splitted', 14),
 ('koboi', 14),
 ('kemprecos', 14),
 ('shoppin', 14),
 ('axillary', 14),
 ('brawlers', 14),
 ('whodunnits', 14),
 ('anquish', 14),
 ('convened', 14),
 ('alsoa', 14),
 ('unopen', 14),
 ('openner', 14),
 ('jomini', 14),
 ('referb', 14),
 ('gimmee', 14),
 ('inteligencia', 14),
 ('fcs', 14),
 ('gumy', 14),
 ('corrals', 14),
 ('baleful', 14),
 ('procedings', 14),
 ('chipettes', 14),
 ('highliter', 14),
 ('perfers', 14),
 ('incrdible', 14),
 ('tullock', 14),
 ('stofen', 1

In [30]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1], reverse = False)[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                counter_object: 29.3 MiB
            least_common_words:  6.8 MiB
            ThreadPoolExecutor:  1.6 KiB
                       Counter:  1.6 KiB
               TfidfVectorizer:  1.6 KiB
                 MultinomialNB:  1.6 KiB
                      Pipeline:  1.6 KiB
                         Dense:  1.6 KiB
                       Dropout:  1.6 KiB
                     Embedding:  1.6 KiB


# Text vectorization

In [31]:
MAX_TOKEN = 100_000
OUTPUT_SEQUENCE_LENGTH = 175  # limiting reviews to 175 words

creating a dictionary of words with counts helps me create text vectorizer alot faster .


it only took 3min 24sec to create the dictionary from 3.6 million entries which is alot faster if you try to
adapt  TextVectorization to the data.

In [32]:
text_vectorizer = TextVectorization(max_tokens=MAX_TOKEN, standardize='lower_and_strip_punctuation',
                                   split='whitespace',
                                    ngrams= None ,
                                    output_mode='int',
                                    output_sequence_length=OUTPUT_SEQUENCE_LENGTH,
                                    pad_to_max_tokens=True,
                                    vocabulary = list(counter_object.keys())[:MAX_TOKEN-2]) # -2 FOR 'PAD' AND 'UNK' TOKEN

In [33]:
random_review = "WHo the duck are you? aren't you a goose?"

In [34]:
word_to_id = tf.keras.layers.StringLookup(vocabulary=text_vectorizer.get_vocabulary(), mask_token='', oov_token='[UNK]')
id_to_word = tf.keras.layers.StringLookup(vocabulary=text_vectorizer.get_vocabulary(), mask_token='', oov_token='[UNK]', invert=True)

In [35]:
clean_text(random_review)

'who the duck are you are not you a goose'

In [37]:
print('random Review: ', random_review)
print('random Review length: ', len(random_review))
print('-------\n')
print('vectorized review: ',text_vectorizer([random_review]))

print('Reverse vectorized review: ',id_to_word(text_vectorizer([random_review])))


print('-------\n')
print('Vocabulary_length: ',len(text_vectorizer.get_vocabulary()))
print('Most frequent words: ',text_vectorizer.get_vocabulary()[:100])
print('least frequent words: ',text_vectorizer.get_vocabulary()[-10:])

random Review:  WHo the duck are you? aren't you a goose?
random Review length:  41
-------

vectorized review:  tf.Tensor(
[[   91     2  8076    58    57     1    57    11 30011     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0 

# Data Preparation

In [36]:

TEST_SIZE = .01 # same as config.TEST_SIZE

In [66]:
xtrain, xtest, ytrain, ytest = train_test_split( df.select('review'), df.select('polarity'), test_size=  .01,  random_state = SEED)

In [67]:
del(df)

In [39]:
print('xtrain shape',xtrain.shape, 'ytrain shape', ytrain.shape)
print('xtest shape',xtest.shape, 'ytest shape', ytest.shape)

xtrain shape (3563999, 1) ytrain shape (3563999, 1)
xtest shape (36000, 1) ytest shape (36000, 1)


# Creating tensorflow dataset using `tf.data` api

In [40]:
BATCH_SIZE = 32

In [41]:
def data_generator(x, y):
    num_samples = len(x)
    for i in range(num_samples):
        yield x[i], y[i]


def create_datasets(x, y, text_vectorizer, batch_size:int = 32, shuffle:bool=True, n_repeat:int = 0, buffer_size:int=1_000_000):

    generator = data_generator(x, y)
    print('Generating...')
    train_dataset = tf.data.Dataset.from_generator(
        lambda: generator,
        output_signature=(
            tf.TensorSpec(shape=(None, x.shape[1]), dtype=tf.string),
            tf.TensorSpec(shape=(None, y.shape[1]), dtype=tf.int32)
        )
    )
    print('Mapping...')
    train_dataset = train_dataset.map(lambda x, y: (tf.cast(text_vectorizer(x), tf.int32)[0], y[0]), tf.data.AUTOTUNE)
    train_dataset = train_dataset.batch(batch_size)

    if shuffle:
        train_dataset = train_dataset.shuffle(buffer_size)

    if n_repeat > 0:
        train_dataset = train_dataset.cache().repeat(n_repeat).prefetch(tf.data.AUTOTUNE)
    elif n_repeat == -1:
        train_dataset = train_dataset.cache().repeat().prefetch(tf.data.AUTOTUNE)
    elif n_repeat == 0:
        train_dataset = train_dataset.cache().prefetch(tf.data.AUTOTUNE)

    return train_dataset


In [42]:
test_dataset = create_datasets(xtest, ytest, text_vectorizer, batch_size=BATCH_SIZE, n_repeat = 0, shuffle=False)

Generating...
Mapping...


In [43]:
train_dataset = create_datasets(xtrain, ytrain, text_vectorizer, batch_size=BATCH_SIZE, n_repeat=2, shuffle=False)


Generating...
Mapping...


In [44]:
# del(xtrain, ytrain)
# del(xtest, ytest, counter_object)

In [ ]:
# train_features = tf.data.Dataset.from_tensor_slices(xtrain)
# train_label = tf.data.Dataset.from_tensor_slices(ytrain)
# test_features = tf.data.Dataset.from_tensor_slices(xtest)
# test_label = tf.data.Dataset.from_tensor_slices(ytest)

# del(xtrain, ytrain, xtest, ytest) # deleting variables to free the memory

In [ ]:
# train_dataset = tf.data.Dataset.zip((train_features, train_label))
# train_dataset = train_dataset.map(lambda x,y: (text_vectorizer(x)[0],y),tf.data.AUTOTUNE)
# train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# test_dataset = tf.data.Dataset.zip((test_features, test_label))
# test_dataset = test_dataset.map(lambda x,y: (text_vectorizer(x)[0],y),tf.data.AUTOTUNE )
# test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# del(train_features, train_label, test_features, test_label) # deleting variables to free the memory

In [46]:
# print('len train dataset: ', len(train_dataset)) # doesn't work for generator generated tensor data
# print('len test dataset: ', len(test_dataset))  # doesn't work for generator generated tensor data


In [45]:
for i,j in train_dataset:
    print(i.shape,j.shape)
    print(i,j)
    break

2023-09-01 15:44:14.149077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string
	 [[{{node Placeholder/_5}}]]
2023-09-01 15:44:14.151450: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype int64
	 [[{{node Placeholder/_6}}]]


(32, 175) (32, 1)
tf.Tensor(
[[ 9220    43  1159 ...     0     0     0]
 [   11   393   201 ...     0     0     0]
 [  120   221   413 ...     0     0     0]
 ...
 [ 1039    20    43 ...     0     0     0]
 [  411 18968     8 ...     0     0     0]
 [    1  1431    75 ...     0     0     0]], shape=(32, 175), dtype=int32) tf.Tensor(
[[1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]], shape=(32, 1), dtype=int32)


2023-09-01 15:44:14.615734: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [46]:
train_length = xtrain.shape[0]// BATCH_SIZE
test_length = xtest.shape[0] // BATCH_SIZE

NameError: name 'xtrain' is not defined

In [49]:
print("train length: ", train_length, '\ntest_length: ', test_length)

train length:  111374 
test_length:  1125


In [47]:
VECTORIZER_PATH = model_dir / 'text_vectorizer.pkl'

if colab:
  VECTORIZER_PATH = save_dir / 'text_vectorizer.pkl'

VECTORIZER_PATH


PosixPath('/home/t/aproject/Amazon-review-sentiment-analysis/model/text_vectorizer.pkl')

In [48]:
# to load text vectorizer
def load_text_vectorizer(vectorizer_path):
    from_disk = pickle.load(open(vectorizer_path, "rb"))
    return TextVectorization.from_config(from_disk['config'])

# Pickle the config and weights
def save_text_vectorizer(vectorizer_path):
    pickle.dump({'config': text_vectorizer.get_config()}
                , open(vectorizer_path, "wb"))


In [49]:
text_vectorizer.vocabulary_size(), text_vectorizer.get_vocabulary()[:4]

(100000, ['', '[UNK]', 'the', 'best'])

In [ ]:
# text_vectorizer = load_text_vectorizer(model_dir/'text_vectorizer.pkl')

In [50]:
ls -la -h ../model

/bin/bash: /home/t/miniconda3/envs/deep/lib/libtinfo.so.6: no version information available (required by /bin/bash)
total 26M
drwxrwxr-x  2 t t 4.0K Sep  1 14:31 ./
drwxrwxr-x 10 t t 4.0K Aug 31 18:50 ../
-rw-rw-r--  1 t t  12M Sep  1 14:30 counter.pkl
-rw-rw-r--  1 t t 9.3M Sep  1 14:30 full_model.h5
-rw-rw-r--  1 t t 3.1M Sep  1 14:30 model_weights.h5
-rw-rw-r--  1 t t 1.1M Sep  1 14:30 text_vectorizer.pkl


In [56]:
save_text_vectorizer(VECTORIZER_PATH)

# Embedding

In [51]:
DIM = 8

In [52]:
embedding = Embedding(input_dim = MAX_TOKEN,output_dim= DIM, mask_zero=True, input_length=OUTPUT_SEQUENCE_LENGTH)
print('Embedded text vectorized random sentence: ',embedding(text_vectorizer(random_review)))

Embedded text vectorized random sentence:  tf.Tensor(
[[ 0.04292144  0.00113931  0.01899529 ... -0.02021828  0.03969901
   0.01261326]
 [-0.01892058 -0.01329989  0.04517606 ... -0.02116247 -0.048376
  -0.03167375]
 [ 0.04843697  0.02070308 -0.04972619 ... -0.01345571 -0.01083864
   0.03726724]
 ...
 [-0.02272482 -0.03220202 -0.02965466 ... -0.00393594 -0.01867509
   0.01087759]
 [-0.02272482 -0.03220202 -0.02965466 ... -0.00393594 -0.01867509
   0.01087759]
 [-0.02272482 -0.03220202 -0.02965466 ... -0.00393594 -0.01867509
   0.01087759]], shape=(175, 8), dtype=float32)


# Model:0 (Naive bayes model)

In [59]:
# model0 = Pipeline([
#     ('tfidf',TfidfVectorizer()),
#     ('multino',MultinomialNB())
# ])

In [60]:
# %%time
# #fit and predict
# model0.fit(xtrain['review'], ytrain)

In [61]:
# pred0 = model0.predict(xtest['review'])

# print(pred0.shape ==  ytest.shape)
# print('pred00.shape: ',pred0.shape)
# print('ytest.shape: ',ytest.shape)

# model0_res = calculate_results(y_true=ytest, y_pred=pred0, model_name='model0: naive bayes')
# print(model0_res)

{'model': 'model0: naive bayes', 'accuracy': 84.79444444444444, 'precision': 0.8482905354316962, 'recall': 0.8479444444444444, 'f1': 0.8479237632137091, 'discription': None}

# Model1

In [53]:

def create_lstm_model(input_shape, max_tokens, dim):
    inputs = keras.Input(shape=(input_shape))
    embedding_layer = Embedding(input_dim=max_tokens, output_dim=dim, mask_zero=True, input_length=input_shape, name='embedding_layer')(inputs)
    x = LSTM(16, return_sequences=True, name = 'lstm_layer_1')(embedding_layer)
    x = LSTM(16, name = 'lstm_layer_2')(x)
    x = Dropout(0.4, name ='dropout_layer')(x)
    x = Dense(64, activation='relu', name = 'dense_layer_1')(x)
    outputs = Dense(1, activation='sigmoid', name = 'dense_layer_2_final')(x)
    return keras.Model(inputs=inputs, outputs=outputs, name='model_lstm')



In [54]:
import tensorflow as tf

class LSTMModel(tf.keras.Model):
    def __init__(self, text_vectorizer, input_shape, max_tokens, dim, dropout_rate=0.4):
        super(CustomLSTMModel, self).__init__()
        self.text_vectorizer = text_vectorizer        
        self.embedding_layer = tf.keras.layers.Embedding(
            input_dim=max_tokens, output_dim=dim, mask_zero=True, input_length=input_shape,
            name='embedding_layer'
        )
        self.lstm_layer_1 = tf.keras.layers.LSTM(16, return_sequences=True, name='lstm_layer_1')
        self.lstm_layer_2 = tf.keras.layers.LSTM(16, name='lstm_layer_2')
        self.dropout_layer = tf.keras.layers.Dropout(dropout_rate, name='dropout_layer')
        self.dense_layer_1 = tf.keras.layers.Dense(64, activation='relu', name='dense_layer_1')
        self.dense_layer_2_final = tf.keras.layers.Dense(1, activation='sigmoid', name='dense_layer_2_final')

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        x = self.lstm_layer_1(x)
        x = self.lstm_layer_2(x)
        x = self.dropout_layer(x)
        x = self.dense_layer_1(x)
        return self.dense_layer_2_final(x)

    def predict_sentiment(self, title, text, preprocess_func=clean_text):
        review = f'{title} {text}' # concatenate the title and text
        clean_review = preprocess_func(review)
        review_sequence = self.text_vectorizer([clean_review])
        prediction = self(review_sequence)
        sentiment_score = prediction[0][0]
        sentiment_label = 'Positive' if sentiment_score >= 0.5 else 'Negative'
        return sentiment_label, sentiment_score
    

# Instantiate the model
output_sequence_length = OUTPUT_SEQUENCE_LENGTH  # actual value
model = LSTMModel(text_vectorizer, input_dim = output_sequence_length, max_tokens=MAX_TOKEN,dim=DIM)
# model = create_lstm_model(output_sequence_length, max_tokens=MAX_TOKEN, dim=DIM)

# Compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['Accuracy'])

In [55]:
model.summary()

Model: "model_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 175)]             0         
                                                                 
 embedding_layer (Embedding)  (None, 175, 8)           800000    
                                                                 
 lstm_layer_1 (LSTM)         (None, 175, 16)           1600      
                                                                 
 lstm_layer_2 (LSTM)         (None, 16)                2112      
                                                                 
 dropout_layer (Dropout)     (None, 16)                0         
                                                                 
 dense_layer_1 (Dense)       (None, 64)                1088      
                                                                 
 dense_layer_2_final (Dense)  (None, 1)                6

In [56]:
# tf.keras.utils.plot_model(model, expand_nested=True, show_shapes=True, show_dtype=True, rankdir='TP')

In [57]:
from tensorflow.keras.callbacks import ModelCheckpoint


In [60]:
# Define the ModelCheckpoint callback
checkpoint_filepath = model_dir / "model_weights.h5"
if colab:
    checkpoint_filepath = save_dir / "model_weights.h5"

model_checkpoint_callback = ModelCheckpoint(checkpoint_filepath, monitor = 'loss', save_best_only=True, save_weights_only=True)

In [61]:
# model = tf.keras.models.load_model(model_dir/'model.h5')
model.load_weights(checkpoint_filepath)

In [71]:
model1 = LSTMModel(text_vectorizer, embed_dim=DIM)

# Compile the model
model1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['Accuracy'])

model1.load_weights(checkpoint_filepath)

ValueError: Unable to load weights saved in HDF5 format into a subclassed Model which has not created its variables yet. Call the Model first, then load the weights.

In [74]:
EPOCHS = 10
steps_per_epoch = int(0.1 * train_length//EPOCHS)
validation_steps = int(1.0 * test_length//EPOCHS)

print('Epochs          : ', EPOCHS)
print('train_length    :',train_length )
print('test_length     :   ',test_length )
print('steps_per_epoch : ',steps_per_epoch)
print('validation_steps: ',validation_steps)

Epochs          :  10
train_length    : 111374
test_length     :    1125
steps_per_epoch :  1113
validation_steps:  112


In [75]:
# %%time

# fit the model
history1 = model.fit(train_dataset, epochs = EPOCHS,
                      validation_data= test_dataset,
                      steps_per_epoch = steps_per_epoch,
                      validation_steps=validation_steps,
                     callbacks=[model_checkpoint_callback]
                    )

Epoch 1/10
1113/1113 [==============================] - 32s 28ms/step - loss: 0.0052 - Accuracy: 0.9986 - val_loss: 0.3441 - val_Accuracy: 0.9343
Epoch 2/10
1113/1113 [==============================] - 30s 27ms/step - loss: 5.6735e-05 - Accuracy: 1.0000 - val_loss: 0.4278 - val_Accuracy: 0.9309
Epoch 3/10
1113/1113 [==============================] - 32s 29ms/step - loss: 1.8509e-05 - Accuracy: 1.0000 - val_loss: 0.4830 - val_Accuracy: 0.9320
Epoch 4/10
1113/1113 [==============================] - 32s 29ms/step - loss: 1.2131e-05 - Accuracy: 1.0000 - val_loss: 0.5372 - val_Accuracy: 0.9326
Epoch 5/10
1113/1113 [==============================] - 35s 32ms/step - loss: 5.5844e-06 - Accuracy: 1.0000 - val_loss: 0.5923 - val_Accuracy: 0.9314
Epoch 6/10
1113/1113 [==============================] - 33s 29ms/step - loss: 1.1391e-05 - Accuracy: 1.0000 - val_loss: 0.6375 - val_Accuracy: 0.9303
Epoch 7/10
1113/1113 [==============================] - 32s 29ms/step - loss: 3.9369e-06 - Accuracy: 1.0

In [ ]:
# model_dir

In [76]:
model.save(save_dir / 'full_model.h5')
# model.load_weights(checkpoint_filepath)

In [77]:
from funcyou.plot import plot_history

#### Evaluation

In [83]:
ypred = tf.squeeze(tf.round(model.predict(test_dataset)))
print('ypred1.shape: ',ypred.shape)


1125/1125 [==============================] - 34s 27ms/step
ypred1.shape:  (36000,)


In [84]:
model_res = calculate_results(ytest,ypred, model_name='model1: LSTM', discription = 'small lstm model with vectorizer and embedding layer')
print(model_res)

{'model': 'model1: LSTM', 'accuracy': 94.0638888888889, 'precision': 0.940657243735739, 'recall': 0.9406388888888889, 'f1': 0.9406394430686046, 'discription': 'small lstm model with vectorizer and embedding layer'}


{'model': 'model1: LSTM', 'accuracy': 94.0638888888889, 'precision': 0.940657243735739, 'recall': 0.9406388888888889, 'f1': 0.9406394430686046, 'discription': 'small lstm model with vectorizer and embedding layer'}


In [69]:
ypred1 = tf.squeeze(tf.round(model1.predict(test_dataset)))
print('ypred1.shape: ',ypred1.shape)


1125/1125 [==============================] - 38s 33ms/step
ypred1.shape:  (36000,)


In [70]:
model1_res = calculate_results(ytest,ypred1, model_name='model1: LSTM', discription = 'small lstm model with vectorizer and embedding layer')
print(model1_res)

{'model': 'model1: LSTM', 'accuracy': 99.66666666666667, 'precision': 0.9966827306204571, 'recall': 0.9966666666666667, 'f1': 0.9966666976237895, 'discription': 'small lstm model with vectorizer and embedding layer'}


In [85]:

def predict_sentiment(title, text, text_vectorizer, lstm_model):
    review = f'{title} {text}' # concatenate the title and text
    clean_review = clean_text(review)
    review_sequence = text_vectorizer([clean_review])
    prediction = lstm_model.predict(review_sequence)
    sentiment_score = prediction[0][0]
    sentiment_label = 'Positive' if sentiment_score >= 0.5 else 'Negative'
    return sentiment_label, sentiment_score


In [86]:
a = "i'm don't , y'll, won't"
clean_text(a)

'i am do not y ll will not'

In [95]:
review_title = "don't be sad"
review_text  =  "man"

sentiment_label, sentiment_score = predict_sentiment(review_title, review_text, text_vectorizer, model)
print(sentiment_label, sentiment_score)

1/1 [==============================] - 0s 27ms/step
Negative 0.04752389


# Final training

In [96]:
history1 = model.fit(test_dataset, epochs = EPOCHS,
                     callbacks=[model_checkpoint_callback]
                    )

Epoch 1/10
1125/1125 [==============================] - 38s 34ms/step - loss: 0.1597 - Accuracy: 0.9438
Epoch 2/10
1125/1125 [==============================] - 30s 26ms/step - loss: 0.1127 - Accuracy: 0.9604
Epoch 3/10
1125/1125 [==============================] - 30s 26ms/step - loss: 0.0852 - Accuracy: 0.9719
Epoch 4/10
1125/1125 [==============================] - 29s 25ms/step - loss: 0.0620 - Accuracy: 0.9813
Epoch 5/10
1125/1125 [==============================] - 29s 26ms/step - loss: 0.0428 - Accuracy: 0.9885
Epoch 6/10
1125/1125 [==============================] - 29s 26ms/step - loss: 0.0291 - Accuracy: 0.9932
Epoch 7/10
1125/1125 [==============================] - 31s 27ms/step - loss: 0.0203 - Accuracy: 0.9954
Epoch 8/10
1125/1125 [==============================] - 28s 25ms/step - loss: 0.0148 - Accuracy: 0.9969
Epoch 9/10
1125/1125 [==============================] - 29s 25ms/step - loss: 0.0108 - Accuracy: 0.9977
Epoch 10/10
1125/1125 [==============================] - 29s 26m

In [97]:
model.save(save_dir / 'full_model.h5')


In [98]:
model.save_weights(checkpoint_filepath)